In [5]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-12-27 15:10:43--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.07s   

2024-12-27 15:10:43 (14.9 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
!pip install -r requirements.txt

In [8]:
with open('input.txt', 'r') as file:
    text = file.read()

print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [9]:
print(len(text))

1115394


In [13]:
chars = sorted(list(set(text)))
print(''.join(chars))
vocab_size = len(chars)
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [53]:
##character level language model

stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}

def encode(s):
    return [stoi[c] for c in s]

def decode(l):
    if isinstance(l, torch.Tensor):
        l = l.tolist()
    if isinstance(l, int):
        return itos[l]
    return ''.join([itos[i] for i in l])

print(encode('hii my name is yash'))
print(decode(encode('hii my name is yash')))



[46, 47, 47, 1, 51, 63, 1, 52, 39, 51, 43, 1, 47, 57, 1, 63, 39, 57, 46]
hii my name is yash


In [54]:
# import tiktoken

# enc = tiktoken.get_encoding('gpt2')

# print(enc.encode('yash'))
# print(enc.decode(enc.encode('yash')))

In [55]:
#encode the shakespeare text

import torch

data = torch.tensor(encode(text), dtype=torch.long) #torch.long is equivalent to torch.int64
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [56]:
#train-val split 90-10
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

print(train_data.shape, val_data.shape)

torch.Size([1003854]) torch.Size([111540])


In [57]:
block_size = 8
decode(train_data[block_size:block_size+1])

't'

In [58]:
x = train_data[:block_size+1]
y = train_data[1:block_size+2]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {decode(context)} the target is {decode(target)}")

when input is F the target is i
when input is Fi the target is r
when input is Fir the target is s
when input is Firs the target is t
when input is First the target is  
when input is First  the target is C
when input is First C the target is i
when input is First Ci the target is t


In [62]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    random_indices = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in random_indices])
    y = torch.stack([data[i+1:i+block_size+1] for i in random_indices])
    return x, y

x, y = get_batch('train')
print(x.shape, y.shape)

torch.Size([4, 8]) torch.Size([4, 8])


In [72]:
import torch.nn as nn
import torch.nn.functional as F


class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        #idx and targets are both (B, T) tensor of integers
        logits = self.token_embedding_table(idx) #B, T, C

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=-1)
        return idx
    
model = BigramLanguageModel(vocab_size)

eval_iters = 20
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            x, y = get_batch(split)
            logits, loss = model(x, y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

logits, loss = model(x, y)

print(logits.shape)
print(loss)

idx = torch.zeros((1, 1), dtype=torch.long) #starting with a newline token
print(decode(model.generate(idx, max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8231, grad_fn=<NllLossBackward0>)

fW.xwni,&HVGYHy3MP;yMDsxZ.!z,v-x$NknAukgY&SYW$Ggs;IYaPc-cTqC !R3fF!

Zg-$wcfCA?WuF:dP-ASfF?:koZHSsJA


In [73]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [76]:
batch_size = 32
for steps in range(100000):
    xb, yb = get_batch('train')

    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(f"loss {loss.item()}")


loss 2.5075936317443848


In [78]:
idx = torch.zeros((1, 1), dtype=torch.long) #starting with a newline token
print(decode(m.generate(idx, max_new_tokens=1000)[0].tolist()))


UKENTUCA:
A: bere-
Herih cave,
Me p th,
PAThagusorthatheve my LIOxffice sks; thintonges man bims Ne.
HAnt prd ffr itass inilfrthyorss ard themert d athambour'shomese ang o's, decowa ato
Th,
BIILisid.
MPor icechatrevand;
Herert isthe ca e mbus!
COLAn pry owouilld
Thay de wir:
Whind w ilound ty, hethalod l, yer tcratem be argh-
NELLIUCLINIsoce s thatllid aigo thod, atemen schak br, awor he ishee my uig witot h thesin than t sheramof akshee y.
Yo, misucar MPOLesthn tibur, s;
Stist bu IO, ny the murunt nard An y turyrinteruthownshe men te
To chanenessar hisolouscourgofterd, r by.
Andonin ipoumemed f f oumy mior s thitartomaveyot ncor'

We the And alers IENGUngis bthods fe h t IOLakndoouco aly t y, stheabet Wo eisondan we.

MNERUMA id ge bupelacosenmur! angrnde

Yory thathe m ISThe 'd n, fau?
t ave RKI sonsioron:
MAn n

Baisheswicu ar meshaloutsewoutyomatheledenomugs; shit s,
ADUENCormalindie oimmbofoun:
Ford, XE:
he the, llimengash hirsowig agimy sertacous st fame t swarwiowhthe th me ie,

In [70]:
import numpy as np

-np.log(1/65)
#this is the expected loss for a random guess, 
# but since we are getting 4.27 it means our model has some extra entropy and the loss is higher than expected


np.float64(4.174387269895637)